In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

## Github Metrics for Stylus Sprint Projects
Query key GitHub activity metrics from 2024-2025 for Stylus Sprint projects, providing insights into project development velocity and community engagement.

In [19]:

query = """
    SELECT  distinct p.display_name as Name,
            m.metric_name as Metric,
            ts.sample_date as Date,
            ts.amount as Value
    FROM metrics_v0 m
    JOIN timeseries_metrics_by_project_v0 ts
    on m.metric_id = ts.metric_id
    JOIN projects_v1 p
    on p.project_id = ts.project_id
    JOIN projects_by_collection_v1 pc
    on p.project_id = pc.project_id
    where metric_name like 'GITHUB_%'
    and YEAR(ts.sample_date) in (2024, 2025)
    and pc.collection_name = 'arb-stylus'
"""

df = client.to_pandas(query)
df['Date'] = pd.to_datetime(df['Date'])

df.to_csv('./data/stylus_github_metrics.csv', index=False)
df

,Name,Metric,Date,Value
0,thirdweb,GITHUB_forks_weekly,2024-01-08,10.0
1,Redstone Finance,GITHUB_forks_weekly,2024-01-08,3.0
2,thirdweb,GITHUB_comments_daily,2024-01-08,30.0
3,Runtime Verification,GITHUB_comments_daily,2024-01-08,18.0
4,Trail of Bits Security Reviews,GITHUB_comments_daily,2024-01-08,3.0
...,...,...,...,...
49606,Debid - Fairblock,GITHUB_active_developers_daily,2024-09-19,1.0
49607,thirdweb,GITHUB_opened_pull_requests_daily,2024-09-19,32.0
49608,Pyth Network,GITHUB_opened_pull_requests_daily,2024-09-19,8.0
49609,Redstone Finance,GITHUB_merged_pull_requests_daily,2024-09-19,4.0


## Mapping the Stylus SDK Ecosystem

This analysis tracks the network of dependencies for projects using the Stylus SDK for Rust. By examining package relationships and ownership patterns, we can understand how the Stylus ecosystem is growing and evolving. 

In [4]:
query = """
WITH seed_repos AS (
  -- First get all repos that depend on stylus-sdk-rs packages
  SELECT DISTINCT 
    sboms.from_artifact_namespace,
    sboms.from_artifact_name
  FROM sboms_v0 sboms
  JOIN package_owners_v0 package_owners
    ON sboms.to_package_artifact_name = package_owners.package_artifact_name
    AND sboms.to_package_artifact_source = package_owners.package_artifact_source
  WHERE package_owners.package_owner_artifact_namespace = 'offchainlabs'
    AND package_owners.package_owner_artifact_name = 'stylus-sdk-rs'
)
SELECT DISTINCT
  sboms.from_artifact_namespace as seed_repo_owner,
  sboms.from_artifact_name as seed_repo_name,
  sboms.to_package_artifact_name as package_name,
  package_owners.package_owner_artifact_namespace as package_repo_owner,
  package_owners.package_owner_artifact_name as package_repo_name,
  sboms.to_package_artifact_source as package_source
FROM sboms_v0 sboms
JOIN package_owners_v0 package_owners
  ON sboms.to_package_artifact_name = package_owners.package_artifact_name
  AND sboms.to_package_artifact_source = package_owners.package_artifact_source
JOIN seed_repos
  ON sboms.from_artifact_namespace = seed_repos.from_artifact_namespace
  AND sboms.from_artifact_name = seed_repos.from_artifact_name
WHERE package_owners.package_owner_artifact_namespace IS NOT NULL
"""
df = client.to_pandas(query)

df.to_csv('./data/stylus-sdk-rs-dependencies.csv', index=False)

df



,seed_repo_owner,seed_repo_name,package_name,package_repo_owner,package_repo_name,package_source
0,theblokc,arbdevbootcamp-reference,crypt,pvorb,node-crypt,NPM
1,fluidity-money,long.so,create-jest,jestjs,jest,NPM
2,offchainlabs,stylus-by-example,regex-automata,rust-lang,regex,RUST
3,cryptosmartnow,bs_arbitrum_main,regex-automata,rust-lang,regex,RUST
4,offchainlabs,stylus-workshop-proxy-update,regex-automata,rust-lang,regex,RUST
...,...,...,...,...,...,...
14525,alt-research,nitro-near,windows-sys,microsoft,windows-rs,RUST
14526,offchainlabs,stylus-erc20,windows-sys,microsoft,windows-rs,RUST
14527,blockscout,cargo-stylus-test-examples,windows-sys,microsoft,windows-rs,RUST
14528,offchainlabs,stylus-chess,windows-sys,microsoft,windows-rs,RUST


## List artifacts of projects in Stylus Sprint
Lookup Github repos for all projects in Stylus sprint

In [ ]:

query = """
    SELECT  p.display_name as title,
            p.project_name as project_name,
            a.artifact_namespace as org
    FROM projects_by_collection_v1 pc
    join projects_v1 p 
    on pc.project_id = p.project_id
    left join int_artifacts_by_project a
    on p.project_id = a.project_id
    and a.artifact_source = 'GITHUB'
    where collection_name = 'arb-stylus'    
    group by 1,2,3
"""

df = client.to_pandas(query)
df.to_csv('./data/project_orgs.csv', index=False)
df

,title,project_name,org
0,Pyth Network,pyth-network,pyth-network
1,WakeUp Labs,wakeuplabs,wakeuplabs
2,Redstone Finance,redstone-finance,redstone-finance
3,Open Source Observer,opensource-observer,opensource-observer
4,Debid - Fairblock,fairblock,fairblock
5,Arbos Foundry,arbos-foundry-bernard-wagner,bernard-wagner
6,Sylow,sylow-warlock-labs,warlock-labs
7,Solang,solang-hyperledger-solang,hyperledger-solang
8,StylusPort,stylusport-teamscanworks,teamscanworks
9,Trail of Bits Security Reviews,trailofbits,trailofbits


## Active Developers in projects in Arbitrum Ecosystem
Lookup projects deploying onchain in Arbitrum and determine monthly active developers 

In [21]:

query = """
    WITH arb_projects AS (
        SELECT distinct project_id
        FROM int_artifacts_by_project_in_ossd
        WHERE artifact_source = 'ARBITRUM_ONE'
        AND artifact_type IN ('CONTRACT', 'FACTORY', 'DEPLOYER')
    )
    SELECT  distinct p.display_name as Name,
            m.metric_name as Metric,
            ts.sample_date as Date,
            ts.amount as Value
    FROM metrics_v0 m
    JOIN timeseries_metrics_by_project_v0 ts
    on m.metric_id = ts.metric_id
    JOIN projects_v1 p
    on p.project_id = ts.project_id
    JOIN arb_projects a
    on p.project_id = a.project_id
    where metric_name = 'GITHUB_active_developers_monthly'
    and YEAR(ts.sample_date) in (2024, 2025)
"""

df = client.to_pandas(query)
df.to_csv('./data/arb_projects_active_dev_monthly.csv', index=False)
df

,Name,Metric,Date,Value
0,powerpool-finance,GITHUB_active_developers_monthly,2024-04-01,3
1,Frax Finance,GITHUB_active_developers_monthly,2024-04-01,3
2,Gravita Protocol,GITHUB_active_developers_monthly,2024-04-01,1
3,MUX Protocol,GITHUB_active_developers_monthly,2024-04-01,1
4,Beefy Finance,GITHUB_active_developers_monthly,2024-04-01,8
...,...,...,...,...
2093,Granary,GITHUB_active_developers_monthly,2025-02-01,1
2094,XY Finance,GITHUB_active_developers_monthly,2025-02-01,2
2095,Punk Domains & Iggy Social,GITHUB_active_developers_monthly,2025-02-01,1
2096,Pump BTC,GITHUB_active_developers_monthly,2025-02-01,1


## Active Developers in projects downstream of Stylus SDK
Lookup projects dependent on Stylus SDK and determine monthly active developers 

In [25]:

query = """
  WITH dep_projects AS (
  SELECT distinct sboms.from_project_id
  FROM sboms_v0 sboms
  JOIN package_owners_v0 package_owners
    ON sboms.to_package_artifact_name = package_owners.package_artifact_name
    AND sboms.to_package_artifact_source = package_owners.package_artifact_source
  WHERE package_owners.package_owner_artifact_namespace = 'offchainlabs'
    AND package_owners.package_owner_artifact_name = 'stylus-sdk-rs'
  )
   SELECT  distinct p.display_name as Name,
            m.metric_name as Metric,
            ts.sample_date as Date,
            ts.amount as Value
    FROM metrics_v0 m
    JOIN timeseries_metrics_by_project_v0 ts
    on m.metric_id = ts.metric_id
    JOIN projects_v1 p
    on p.project_id = ts.project_id
    JOIN dep_projects a
    on p.project_id = a.from_project_id
    where metric_name = 'GITHUB_active_developers_monthly'
    and YEAR(ts.sample_date) in (2024, 2025)
"""

df = client.to_pandas(query)
df.to_csv('./data/stylus_dependencies_active_dev_monthly.csv', index=False)
df


,Name,Metric,Date,Value
0,Fracton Research,GITHUB_active_developers_monthly,2024-01-01,1
1,Fracton Research,GITHUB_active_developers_monthly,2024-01-01,1
2,CryptoSmartNow,GITHUB_active_developers_monthly,2025-02-01,1
3,CryptoSmartNow,GITHUB_active_developers_monthly,2025-02-01,1
4,CryptoSmartNow,GITHUB_active_developers_monthly,2024-03-01,3
...,...,...,...,...
291,LimeChain,GITHUB_active_developers_monthly,2024-05-01,15
292,LimeChain,GITHUB_active_developers_monthly,2024-09-01,12
293,LimeChain,GITHUB_active_developers_monthly,2024-09-01,12
294,LimeChain,GITHUB_active_developers_monthly,2024-04-01,14
